Grupo 3

In [ ]:
import numpy as np

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{I} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.

In [1]:
import numpy as np
from typing import Union

def householder(x: np.ndarray) -> Union[np.ndarray, int]:
    alpha = x[0]
    s = np.power(np.linalg.norm(x[1:]), 2)
    v = x.copy()

    if s == 0:
        tau = 0
    else:
        t = np.sqrt(alpha**2 + s)
        v[0] = alpha - t if alpha <= 0 else -s / (alpha + t)

        tau = 2 * v[0]**2 / (s + v[0]**2)
        v /= v[0]

    return v, tau

def qr_decomposition(A: np.ndarray) -> Union[np.ndarray, np.ndarray]:
    m, n = A.shape
    R = A.copy()
    Q = np.identity(m)

    for j in range(n):
        v, tau = householder(R[j:, j])
        H = np.identity(m)
        H[j:, j:] -= tau * np.outer(v, v)  # Outer product!
        R = H @ R
        Q = H @ Q.T  # Transpose!

    return Q.T, R

m = 10
n = 8

A = np.random.rand(m, n)
q, r = np.linalg.qr(A)
Q, R = qr_decomposition(A)

print("*****")
print(Q)
print(q)
print("-----")
print(R)
print(r)


*****
[[ 4.09818640e-01 -3.20548449e-01 -3.59980217e-02  1.14332833e-01
   3.47498251e-01 -9.36079961e-02  4.55125157e-01  1.18360556e-01
   4.11420231e-02  6.02138882e-01]
 [ 1.99995578e-01  4.50787786e-01 -3.81265345e-01  3.28106920e-01
   1.53226872e-01 -3.33159484e-01 -4.86010282e-02 -1.91000773e-02
  -6.04151549e-01 -3.96868248e-02]
 [-2.35082411e-01  8.82663456e-02  6.59623724e-01  3.85859354e-01
   2.70417843e-01  7.17936984e-02  5.33601143e-02  4.71580894e-01
  -2.02913963e-01 -9.09084779e-02]
 [-9.72887912e-02  4.17180712e-01 -6.89044459e-02 -3.71225448e-01
   1.14381142e-01  3.44588651e-03 -4.36249438e-01  4.16752923e-01
   1.35308493e-01  5.27768045e-01]
 [-5.24339849e-01 -2.31245244e-01 -5.19594812e-01  4.47888533e-01
   1.13851105e-01  3.82739246e-01 -9.01413155e-02  7.60428450e-02
   3.70394559e-02  1.62108469e-01]
 [ 1.96190755e-02  4.60175869e-01  1.81718684e-02  4.74037746e-01
  -8.75049250e-02 -2.21641531e-01  1.39290471e-01 -1.20278219e-01
   6.86475468e-01  3.019813

Test your function using tests below:

In [ ]:
import numpy as np

def householder(v):
    sigma = np.sign(v[0]) * np.linalg.norm(v, ord=2)
    v[0] += sigma
    v /= np.linalg.norm(v, ord=2)
    H = np.eye(len(v)) - 2 * np.outer(v, v)
    return v, H

# Test I.1
v = np.array([1, 2, 3])
v1, h = householder(v)

np.testing.assert_allclose(np.dot(h, v1), v)
np.testing.assert_allclose(np.dot(h, v), v1)

In [10]:
import numpy as np

def householder(v):
    n = len(v)
    v_norm = np.linalg.norm(v)
    if v_norm == 0:
        raise ValueError('Cannot normalize zero vector')
    v1 = np.zeros(n)
    v1[0] = np.sign(v[0]) * v_norm
    u = v - v1
    h = np.eye(n) - 2 * np.outer(u, u) / np.dot(u, u)
    return v1, h

# Test I.2
rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

np.testing.assert_allclose(np.dot(h, v1), vec)


# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

$$
\mathbf{A} = (\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1)^{-1} \mathbf{R} \;,
$$
so 
$$
\mathbf{A} =  \mathbf{Q} \mathbf{R} \;,
$$
with
$$
\mathbf{Q} = (\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1)^{-1} =  \mathbf{H}_1^{-1}  \mathbf{H}_2^{-1}  \cdots \mathbf{H}_{n-1}^{-1} = \mathbf{H}_1^T  \mathbf{H}_2^T  \cdots \mathbf{H}_{n-1}^T 
$$
Since $\mathbf{H}_i$ is ortogonal
$$
\mathbf{H}_i\mathbf{H}_i^T = \mathbf{I}
$$
then
$$
\mathbf{H}^{-1} = \mathbf{H}^T 
$$
but also  $\mathbf{H}_i$ is symetric
$$
\mathbf{H}_i^T = \mathbf{H}_i
$$
so
$$
\mathbf{Q} = \mathbf{H}_1^{-1}  \mathbf{H}_2^{-1}  \cdots \mathbf{H}_{n-1}^{-1} = \mathbf{H}_1^T  \mathbf{H}_2^T  \cdots \mathbf{H}_{n-1}^T =  \mathbf{H}_1 \mathbf{H}_2  \cdots \mathbf{H}_{n-1}
$$


In [18]:
import numpy as np

def qr_decomp(a):
    m, n = a.shape
    Q = np.eye(m)
    R = a.copy()

    for j in range(n):
        x = R[j:, j]
        e = np.zeros_like(x)
        e[0] = np.sign(x[0])

        v = np.linalg.norm(x) * e + x
        v = v / np.linalg.norm(v)

        R[j:, :] -= 2 * np.outer(v, np.dot(v, R[j:, :]))
        Q[:, j:] -= 2 * np.outer(Q[:, j:], np.dot(Q[:, j:].T, v))

    return Q, R

In [12]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [ ]:
# Test II.1 (20% of the total grade)
rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)
# test that Q is indeed orthogonal
np.testing.assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
np.testing.assert_allclose(np.dot(q, r), a)

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [ ]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

Check if your q and r agree with qq and rr. Explain.

In [ ]:
from scipy.linalg import qr
a = np.array([[4,3,1], [5,7,0], [9,9,3], [8,2,4], [9,3,1]])
q, r = qr_decomp(a)
qq, rr = qr(a)
print(q)
print(qq)
print(r)
print(rr)
print(a)
print(q@r)
print(qq@rr)

Enter your explanation here (10% of the total grade, peer-graded)

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).





Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests. 

(Peer-graded, 40% of the total grade)

$$
\mathbf{R} = \mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} 
$$
and 
$$
\mathbf{H}_i = \mathbf{I} - 2 \mathbf{u}_i \mathbf{u}_i^T
$$
so if
$$
\mathbf{R}_0 =  \mathbf{A}
$$
then 
$$
\mathbf{R}_1 = \mathbf{H}_1 \mathbf{R}_0= \ (\mathbf{I} - 2 \mathbf{u}_1 \mathbf{u}_1^T) \mathbf{R}_0 =  \mathbf{R}_0 -  2 \mathbf{u}_1 ( \mathbf{u}_1^T  \mathbf{R}_0)
$$
and
$$
\mathbf{R}_2 = \mathbf{H}_2 \mathbf{R}_1 = \ (\mathbf{I} - 2 \mathbf{u}_2 \mathbf{u}_2^T) \mathbf{R}_1 =  \mathbf{R}_1 -  2 \mathbf{u}_2 ( \mathbf{u}_2^T  \mathbf{R}_1)
$$
so on until
$$
\mathbf{R} = \mathbf{H}_n \mathbf{R}_{n-1} =  (\mathbf{I} - 2 \mathbf{u}_{n-1} \mathbf{u}_{n-1}^T) \mathbf{R}_{n-1} = \mathbf{R}_{n-1} -  2 \mathbf{u}_{n-1} ( \mathbf{u}_{n-1}^T  \mathbf{R}_{n-1})
$$

In [24]:
import numpy as np

def r_decomp(a):
    m, n = a.shape
    R = a.copy()
    vecs = np.zeros((m, n))

    for j in range(n):
        x = R[j:, j]
        norm_x = np.linalg.norm(x)
        sign_x_1 = np.sign(x[0])

        vec = np.copy(x)
        vec[0] += sign_x_1 * norm_x
        vec /= np.linalg.norm(vec)

        vecs[j:, j] = vec
        
        R[j:, j:] -= 2 * np.outer(vec, np.dot(vec, R[j:, j:]))

    return R, vecs


$$
\mathbf{Q} =  \mathbf{H}_1 \mathbf{H}_2  \cdots \mathbf{H}_{n-1}
$$
and
$$
\mathbf{H}_i = \mathbf{I} - 2 \mathbf{u}_i \mathbf{u}_i^T
$$
so 
$$
\mathbf{Q} =  (\mathbf{I} - 2 \mathbf{u}_1 \mathbf{u}_1^T )( \mathbf{I} - 2 \mathbf{u}_2 \mathbf{u}_2^T)  \cdots (\mathbf{I} - 2 \mathbf{u}_{n-1} \mathbf{u}_{n-1}^T)
$$

In [28]:
import numpy as np

def q_decomp(vecs):
    m, n = vecs.shape
    Q = np.eye(m)

    for j in range(n):
        vec = vecs[:, j]
        Q -= 2*np.outer(vec, np.dot(vec, Q))

    QT = np.linalg.inv(Q)
    
    return Q, QT


In [ ]:
rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
r, vecs = r_decomp(a)
q,qt = q_decomp(vecs)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

In [ ]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

In [ ]:
a = np.array([[4,3,1], [5,7,0], [9,9,3], [8,2,4], [9,3,1]])
R, vecs = r_decomp(a)
print("R\n ",R)
Q,QT = q_decomp(vecs)
print("Q\n",Q)
print("QT\n",QT)
print("QT@Q\n",QT@Q)
print("a\n",a)
print("Q@R\n",Q@R)

R
  [[16.34013464 10.46503005  4.40632844]
 [ 0.          6.51790964 -0.7843557 ]
 [ 0.         -0.          2.63989693]
 [ 0.          0.          0.        ]
 [ 0.         -0.         -0.        ]]
Q
 [[ 0.24479602  0.06723049 -0.00981821  0.96611775  0.04555281]
 [ 0.30599503  0.58266423 -0.33762629 -0.09006618 -0.6669078 ]
 [ 0.55079106  0.4964713   0.36457628 -0.19531237  0.52828277]
 [ 0.48959205 -0.47923271  0.55563068 -0.06304632 -0.46683546]
 [ 0.55079106 -0.42406923 -0.66653641 -0.12799536  0.23694072]]
QT
 [[ 0.24479602  0.30599503  0.55079106  0.48959205  0.55079106]
 [ 0.06723049  0.58266423  0.4964713  -0.47923271 -0.42406923]
 [-0.00981821 -0.33762629  0.36457628  0.55563068 -0.66653641]
 [ 0.96611775 -0.09006618 -0.19531237 -0.06304632 -0.12799536]
 [ 0.04555281 -0.6669078   0.52828277 -0.46683546  0.23694072]]
QT@Q
 [[ 1. -0.  0.  0. -0.]
 [-0.  1. -0. -0.  0.]
 [ 0. -0.  1.  0. -0.]
 [ 0. -0.  0.  1.  0.]
 [-0.  0. -0.  0.  1.]]
a
 [[4 3 1]
 [5 7 0]
 [9 9 3]
 [8 2 4]
